In [121]:
import os
import re
from collections import defaultdict

In [156]:
dir_path = "../projects_with_docker"

In [158]:
def preprocess_dockerfile(dockerfile_content):
    """
    Preprocess the Dockerfile to combine lines split by backslashes.
    """
    lines = dockerfile_content.splitlines()
    combined_lines = []
    buffer = ""

    for line in lines:
        stripped_line = line.strip()
        if stripped_line.endswith('\\'):
            # Remove the backslash and append to the buffer
            buffer += stripped_line[:-1].strip() + " "
        else:
            # Add the remaining part of the line to the buffer and save it
            buffer += stripped_line
            combined_lines.append(buffer)
            buffer = ""  # Reset the buffer

    return combined_lines

def parse_dockerfile(dockerfile_content):
    """
    Parse a Dockerfile and extract commands and their arguments.
    Returns a dictionary where keys are commands and values are lists of arguments.
    """
    commands = defaultdict(list)
    combined_lines = preprocess_dockerfile(dockerfile_content)

    for line in combined_lines:
        line = line.strip()
        if not line or line.startswith('#'):
            continue  # Skip empty lines and comments

        # Match Dockerfile commands (e.g., FROM, RUN, ENV, etc.)
        match = re.match(r'^\s*(\w+)\s+(.*)$', line, re.IGNORECASE)
        if match:
            command = match.group(1).upper()
            args = match.group(2).strip()

            # Handle FROM command
            if command == 'FROM':
                # Extract the base image name (ignore aliases and additional arguments)
                base_image = args.split()[0]  # Get the first part (e.g., "python:3.8")
                commands[command].append(base_image)
            # Handle multi-instruction RUN commands
            elif command == 'RUN':
                # Split by '&&' and strip whitespace
                args_list = [a.strip() for a in args.split('&&') if a.strip()]
                commands[command].extend(args_list)
            # Handle multi-file COPY commands
            elif command == 'COPY':
                # Split into source files and destination
                parts = args.split()
                if len(parts) >= 2:
                    destination = parts[-1]
                    sources = parts[:-1]
                    for source in sources:
                        commands[command].append(f"{source} -> {destination}")
            # Handle ENV, ARG, and LABEL commands (key-value pairs)
            elif command in ['ENV', 'ARG', 'LABEL']:
                # Split into key-value pairs
                pairs = re.findall(r'(\S+)=\s*("[^"]*"|\S+)', args)
                for key, value in pairs:
                    commands[command].append(f"{key}={value}")
            # Handle EXPOSE and VOLUME commands (space-separated lists)
            elif command in ['EXPOSE', 'VOLUME']:
                # Split into individual items
                items = args.split()
                commands[command].extend(items)
            else:
                commands[command].append(args)

    return commands

def calculate_precision_recall(generated, manual):
    """
    Calculate precision and recall for each command type.
    Returns a dictionary with counts and actual items for TP, FP, and FN.
    """
    results = {}

    # Get all unique commands
    all_commands = set(generated.keys()).union(set(manual.keys()))

    for command in all_commands:
        generated_args = set(generated.get(command, []))
        manual_args = set(manual.get(command, []))

        # True positives: commands present in both generated and manual
        tp_items = generated_args.intersection(manual_args)
        tp = len(tp_items)

        # False positives: commands in generated but not in manual
        fp_items = generated_args - manual_args
        fp = len(fp_items)

        # False negatives: commands in manual but not in generated
        fn_items = manual_args - generated_args
        fn = len(fn_items)

        # Precision: TP / (TP + FP)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0

        # Recall: TP / (TP + FN)
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        results[command] = {
            'precision': precision,
            'recall': recall,
            'tp': tp,
            'fp': fp,
            'fn': fn,
            'tp_items': list(tp_items),  # Actual items for TP
            'fp_items': list(fp_items),  # Actual items for FP
            'fn_items': list(fn_items)   # Actual items for FN
        }

    return results

def find_dockerfiles(root_dir):
    """
    Recursively find all Dockerfile, Dockerfile_MiDKo_topic, and Dockerfile_MiDKo files.
    Returns a list of tuples: (dockerfile_path, midko_topic_path, midko_path).
    """
    dockerfile_tuples = []

    for dirpath, _, filenames in os.walk(root_dir):
        dockerfile_path = None
        midko_topic_path = None
        midko_path = None

        for filename in filenames:
            if filename == 'Dockerfile':
                dockerfile_path = os.path.join(dirpath, filename)
            elif filename == 'Dockerfile_MiDKo_topic':
                midko_topic_path = os.path.join(dirpath, filename)
            elif filename == 'Dockerfile_MiDKo':
                midko_path = os.path.join(dirpath, filename)

        # If all three files are found, add them to the list
        if dockerfile_path and midko_topic_path and midko_path:
            dockerfile_tuples.append((dockerfile_path, midko_topic_path, midko_path))

    return dockerfile_tuples

In [160]:
def process_directory(root_dir):
    """
    Process all Dockerfiles in the root directory and compare them.
    Returns precision, recall, and counts of TP, FP, and FN for each command in each file,
    as well as overall mean precision and recall, and total precision and recall.
    """
    # Initialize a list to store results for each file
    file_results = []

    # Initialize dictionaries to store precision and recall for each command across all files
    precision_dict_topic = defaultdict(list)
    recall_dict_topic = defaultdict(list)

    precision_dict_midko = defaultdict(list)
    recall_dict_midko = defaultdict(list)

    # Initialize totals for TP, FP, and FN across all files
    total_tp_topic = 0
    total_fp_topic = 0
    total_fn_topic = 0

    total_tp_midko = 0
    total_fp_midko = 0
    total_fn_midko = 0

    # Find all Dockerfile tuples
    dockerfile_tuples = find_dockerfiles(root_dir)

    for dockerfile_path, midko_topic_path, midko_path in dockerfile_tuples:
        # Read Dockerfiles
        with open(dockerfile_path, 'r') as f:
            dockerfile_content = f.read()
        with open(midko_topic_path, 'r') as f:
            midko_topic_content = f.read()
        with open(midko_path, 'r') as f:
            midko_content = f.read()

        # Parse Dockerfiles
        dockerfile_commands = parse_dockerfile(dockerfile_content)
        midko_topic_commands = parse_dockerfile(midko_topic_content)
        midko_commands = parse_dockerfile(midko_content)

        # Compare Dockerfile with Dockerfile_MiDKo_topic
        results_topic = calculate_precision_recall(midko_topic_commands, dockerfile_commands)
        # Compare Dockerfile with Dockerfile_MiDKo
        results_midko = calculate_precision_recall(midko_commands, dockerfile_commands)

        # Store results for this file
        file_results.append({
            'dockerfile_path': dockerfile_path,
            'midko_topic_path': midko_topic_path,
            'midko_path': midko_path,
            'results_topic': results_topic,  # Results for Dockerfile_MiDKo_topic
            'results_midko': results_midko   # Results for Dockerfile_MiDKo
        })

        # Store precision and recall for each command across all files
        for command in results_topic:
            precision_dict_topic[command].append(results_topic[command]['precision'])
            recall_dict_topic[command].append(results_topic[command]['recall'])
            # Aggregate TP, FP, and FN across all files
            total_tp_topic += results_topic[command]['tp']
            total_fp_topic += results_topic[command]['fp']
            total_fn_topic += results_topic[command]['fn']
        for command in results_midko:
            precision_dict_midko[command].append(results_midko[command]['precision'])
            recall_dict_midko[command].append(results_midko[command]['recall'])
            # Aggregate TP, FP, and FN across all files
            total_tp_midko += results_midko[command]['tp']
            total_fp_midko += results_midko[command]['fp']
            total_fn_midko += results_midko[command]['fn']

    # Calculate overall mean precision and recall for Dockerfile_MiDKo_topic
    overall_precision_topic = {
        command: sum(precision_dict_topic[command]) / len(precision_dict_topic[command])
        for command in precision_dict_topic
    }
    overall_recall_topic = {
        command: sum(recall_dict_topic[command]) / len(recall_dict_topic[command])
        for command in recall_dict_topic
    }

    # Calculate overall mean precision and recall for Dockerfile_MiDKo
    overall_precision_midko = {
        command: sum(precision_dict_midko[command]) / len(precision_dict_midko[command])
        for command in precision_dict_midko
    }
    overall_recall_midko = {
        command: sum(recall_dict_midko[command]) / len(recall_dict_midko[command])
        for command in recall_dict_midko
    }

    # Calculate total precision and recall for Dockerfile_MiDKo_topic
    total_precision_topic = total_tp_topic / (total_tp_topic + total_fp_topic) if (total_tp_topic + total_fp_topic) > 0 else 1.0
    total_recall_topic = total_tp_topic / (total_tp_topic + total_fn_topic) if (total_tp_topic + total_fn_topic) > 0 else 1.0

    # Calculate total precision and recall for Dockerfile_MiDKo
    total_precision_midko = total_tp_midko / (total_tp_midko + total_fp_midko) if (total_tp_midko + total_fp_midko) > 0 else 1.0
    total_recall_midko = total_tp_midko / (total_tp_midko + total_fn_midko) if (total_tp_midko + total_fn_midko) > 0 else 1.0

    return (
        file_results,
        overall_precision_topic,
        overall_recall_topic,
        overall_precision_midko,
        overall_recall_midko,
        total_precision_topic,
        total_recall_topic,
        total_precision_midko,
        total_recall_midko
    )

def main(root_dir):
    # Process all files and get precision, recall, and counts for each command in each file
    (
        file_results,
        overall_precision_topic,
        overall_recall_topic,
        overall_precision_midko,
        overall_recall_midko,
        total_precision_topic,
        total_recall_topic,
        total_precision_midko,
        total_recall_midko
    ) = process_directory(root_dir)

    # Print results for each file
    # print("Comparison Results for Each File:")
    # for result in file_results:
    #     print(f"File: {result['dockerfile_path']}")
    #     print(f"  Dockerfile_MiDKo_topic:")
    #     for command in sorted(result['results_topic'].keys()):
    #         metrics = result['results_topic'][command]
    #         print(f"    Command: {command}")
    #         print(f"      True Positives (TP): {metrics['tp']}")
    #         print(f"      False Positives (FP): {metrics['fp']}")
    #         print(f"      False Negatives (FN): {metrics['fn']}")
    #         print(f"      Precision: {metrics['precision']:.2f}")
    #         print(f"      Recall: {metrics['recall']:.2f}")
    #     print(f"  Dockerfile_MiDKo:")
    #     for command in sorted(result['results_midko'].keys()):
    #         metrics = result['results_midko'][command]
    #         print(f"    Command: {command}")
    #         print(f"      True Positives (TP): {metrics['tp']}")
    #         print(f"      False Positives (FP): {metrics['fp']}")
    #         print(f"      False Negatives (FN): {metrics['fn']}")
    #         print(f"      Precision: {metrics['precision']:.2f}")
    #         print(f"      Recall: {metrics['recall']:.2f}")
    #     print()

    # Print overall mean precision and recall for Dockerfile_MiDKo_topic
    print("Overall Mean Precision and Recall for Dockerfile_MiDKo_topic:")
    for command in sorted(overall_precision_topic.keys()):
        print(f"  Command: {command}")
        print(f"    Mean Precision: {overall_precision_topic[command]:.2f}")
        print(f"    Mean Recall: {overall_recall_topic[command]:.2f}")
    print()

    # Print overall mean precision and recall for Dockerfile_MiDKo
    print("Overall Mean Precision and Recall for Dockerfile_MiDKo:")
    for command in sorted(overall_precision_midko.keys()):
        print(f"  Command: {command}")
        print(f"    Mean Precision: {overall_precision_midko[command]:.2f}")
        print(f"    Mean Recall: {overall_recall_midko[command]:.2f}")
    print()

    # Print total precision and recall for Dockerfile_MiDKo_topic
    print("Total Precision and Recall for Dockerfile_MiDKo_topic:")
    print(f"  Total Precision: {total_precision_topic:.2f}")
    print(f"  Total Recall: {total_recall_topic:.2f}")
    print()

    # Print total precision and recall for Dockerfile_MiDKo
    print("Total Precision and Recall for Dockerfile_MiDKo:")
    print(f"  Total Precision: {total_precision_midko:.2f}")
    print(f"  Total Recall: {total_recall_midko:.2f}")


In [ ]:
main(dir_path)